# Tutorial 3.4: Doing PDSketch with STRIPS-Style Heuristics

In [1]:
import concepts.pdsketch as pds

In [2]:
# From tutorial/3-pdsketch/3-translate-into-strips.ipynb
domain_string = r"""(define (domain blocks-wold)
    (:types block)
    (:predicates
        (clear ?x - block)          ;; no block is on x
        (on ?x - block ?y - block)  ;; x is on y
        (robot-holding ?x - block)  ;; the robot is holding x
        (robot-handfree)            ;; the robot is not holding anything
    )
    (:action pick
     :parameters (?x - block)
     :precondition (and (robot-handfree) (clear ?x))
     :effect (and (not (robot-handfree)) (robot-holding ?x) (not (clear ?x)))
    )
    (:action place
     :parameters (?x - block ?y - block)
     :precondition (and (robot-holding ?x) (clear ?y))
     :effect (and (robot-handfree) (not (robot-holding ?x)) (not (clear ?y)) (clear ?x) (on ?x ?y))
    )
)"""

In [3]:
domain = pds.load_domain_string(domain_string)
domain

Domain(blocks-wold)

In [4]:
executor = pds.PDSketchExecutor(domain)

In [5]:
# From tutorial/3-pdsketch/3-translate-into-strips.ipynb
state, ctx = executor.new_state({'a': domain.types['block'], 'b': domain.types['block'], 'c': domain.types['block']}, create_context=True)
ctx.define_predicates([
    ctx.robot_handfree(),
    ctx.clear('a'),
    ctx.clear('b'),
    ctx.clear('c')
])
state

State{
  states:
    - on: Value[bool, axes=[?x, ?y], tdtype=torch.int64, tdshape=(3, 3), quantized]{
      tensor([[0, 0, 0],
              [0, 0, 0],
              [0, 0, 0]])
    }
    - robot-holding: Value[bool, axes=[?x], tdtype=torch.int64, tdshape=(3,), quantized]{tensor([0, 0, 0])}
    - clear: Value[bool, axes=[?x], tdtype=torch.int64, tdshape=(3,), quantized]{tensor([1, 1, 1])}
    - robot-handfree: Value[bool, axes=[], tdtype=torch.int64, tdshape=(), quantized]{tensor(1)}
  objects: a - block, b - block, c - block
}

In [6]:
goal_expr = domain.parse('(and (on a b) (on b c))')
goal_expr

AndExpression<and(on(OBJ::a, OBJ::b), on(OBJ::b, OBJ::c))>

In [7]:
from concepts.pdsketch.planners.discrete_search import heuristic_search_strips
plan = heuristic_search_strips(executor, state, goal_expr, 'hff', verbose=True)
plan

hsstrips::actions nr 12
hsstrips::goal_expr and(on(OBJ::a, OBJ::b), on(OBJ::b, OBJ::c))


heuristic_search::expanding: priority = 0 g = 4: : 6it [00:00, 513.28it/s]

hsstrips::search succeeded.
hsstrips::total_expansions: 6


(OperatorApplier<action::pick(?x=b)>,
 OperatorApplier<action::place(?x=b, ?y=c)>,
 OperatorApplier<action::pick(?x=a)>,
 OperatorApplier<action::place(?x=a, ?y=b)>)